In [2]:
import sys
import os
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import grad

# device = torch.device('cuda')
device = torch.device('cpu')

sys.path.append('utils_continuous')
from utils_continuous import preprocess_data_continuous_inference, plot_results_continuous_inference

In [3]:
# Load the dataset and preprocess it
data_path = 'data/burgers_shock.mat'

Nu = 100
Nf = 10000

x, t, u_exact, X, T, lb, ub, train_X_u, train_u, train_X_f, test_X, test_u = preprocess_data_continuous_inference(data_path, Nu, Nf)
train_X_u = torch.tensor(train_X_u, dtype = torch.float, requires_grad = True, device = device)
train_u = torch.tensor(train_u, dtype = torch.float, requires_grad = True, device = device)
train_X_f = torch.tensor(train_X_f, dtype = torch.float, requires_grad = True, device = device)
test_X = torch.tensor(test_X, dtype = torch.float, requires_grad = True, device = device)
test_u = torch.tensor(test_u, dtype = torch.float, requires_grad = True, device = device)

In [4]:
class NeuralNet(nn.Module):

    def __init__(self, layers, lb, ub):
        super().__init__()
        self.lb = torch.tensor(lb, dtype = torch.float, device = device)
        self.ub = torch.tensor(ub, dtype = torch.float, device = device)
        
        # Layer module
        self.layers = nn.ModuleList()
        
        # Make the neural network
        input_dim = layers[0]
        for output_dim in layers[1:]:
            self.layers.append(nn.Linear(input_dim, output_dim))
            nn.init.xavier_normal_(self.layers[-1].weight)
            input_dim = output_dim
        
        
    def forward(self, X):
        x = 2.0*(X - self.lb)/(self.ub - self.lb) - 1.0
        for layer in self.layers[:-1]:
            x = torch.tanh(layer(x))

        outputs = self.layers[-1](x)
        return outputs

In [5]:
class PINN:
    def __init__(self, X_f, lb, ub, nu, layers = [2, 20, 20, 20, 1], lr = 1e-2, device = torch.device('cpu')):
        self.nu = nu
        self.x_f = X_f[:, :1]
        self.t_f = X_f[:, 1:]
        
        self.net_u = NeuralNet(layers, lb, ub)
        self.net_u.to(device)
        
        self.optimizer = torch.optim.Adam(self.net_u.parameters(), lr = lr, betas=(0.9, 0.999))
    
    def net_f(self):
        
        u = self.net_u(torch.cat([self.x_f, self.t_f],1))
        
        u_temp1 = torch.ones(self.x_f.size(), device = device)
        u_temp2 = torch.ones(self.x_f.size(), device = device)
        u_temp3 = torch.ones(self.x_f.size(), device = device)
        
        u_t = grad(u, self.t_f, grad_outputs = u_temp1, create_graph = True)[0]
        u_x = grad(u, self.x_f, grad_outputs = u_temp2, create_graph = True)[0]
        u_xx = grad(u_x, self.x_f, grad_outputs = u_temp3, create_graph = True)[0]
        
        f = u_t + u*u_x - (self.nu)*u_xx
        
        del u_temp1
        del u_temp2
        del u_temp3
        
        return f
    
    def loss_f(self, u, u_pred):
        return torch.mean(torch.square(u-u_pred)) + torch.mean(torch.square(self.net_f()))
    
    def optimizer_step(self, X_u, u):
        # Zero the grads for the model paramters in the optimizer
        self.optimizer.zero_grad()
            
        # Compute the losses and backpropagate the losses
        loss = self.loss_f(u, self.net_u(X_u))
        loss.backward()

        # Optimizer one iteration with the given optimizer
        self.optimizer.step()
        
        return loss.item()
    
    def fit(self, X_u, u, epochs = 1):
        for epoch in range(epochs):
            loss_value = self.optimizer_step(X_u, u)

            if epoch % 100 == 99:
                with torch.no_grad():
                    valid_loss = torch.norm(test_u - pinn.predict(test_X), 2)/torch.norm(test_u, 2)
                    print(f'Epoch {epoch+1}: Training Loss = {loss_value}, Validation Loss = {valid_loss.item()}')
    
    def predict(self, test_X):
        return self.net_u(test_X)

In [12]:
# Define the model
pinn = PINN(train_X_f, lb, ub, nu = (0.01/np.pi), layers = [2, 20, 20, 20, 20, 20, 20, 20, 20, 1], lr = 0.001, device = device)

In [13]:
# Train the model
pinn.fit(train_X_u, train_u, epochs = 3000)

Epoch 100: Training Loss = 0.09432919323444366, Validation Loss = 0.5668474435806274
Epoch 200: Training Loss = 0.06752654165029526, Validation Loss = 0.509410560131073
Epoch 300: Training Loss = 0.05274968966841698, Validation Loss = 0.4611217677593231
Epoch 400: Training Loss = 0.03988169878721237, Validation Loss = 0.39974337816238403
Epoch 500: Training Loss = 0.029588177800178528, Validation Loss = 0.32834091782569885
Epoch 600: Training Loss = 0.021516047418117523, Validation Loss = 0.2844264805316925
Epoch 700: Training Loss = 0.01302124559879303, Validation Loss = 0.2292969673871994
Epoch 800: Training Loss = 0.013239944353699684, Validation Loss = 0.4802347719669342
Epoch 900: Training Loss = 0.00993889570236206, Validation Loss = 0.48654472827911377
Epoch 1000: Training Loss = 0.004674773663282394, Validation Loss = 0.298623651266098
Epoch 1100: Training Loss = 0.0036241125781089067, Validation Loss = 0.28950515389442444
Epoch 1200: Training Loss = 0.0027265315875411034, Vali

In [14]:
# Evaluate the model
u_pred = pinn.predict(test_X)
valid_loss = torch.norm(test_u - pinn.predict(test_X), 2)/torch.norm(test_u, 2)
print(valid_loss.item())

0.09546840190887451


In [15]:
# Plot the results
%matplotlib widget
device = torch.device('cpu')

plot_results_continuous_inference(x, t, X, T, u_exact, u_pred.to(device).detach().numpy(),
                                  train_X_u.to(device).detach().numpy(), train_X_f.to(device).detach().numpy(),
                                  train_u.to(device).detach().numpy(), test_X.to(device).detach().numpy())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …